In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import platform
#import SIRD_Model_Experimental
import SIRD_Model
from sklearn import linear_model


In [11]:
def getGamma(infect, dead):
    
    recovG = approxRecovered(infect)
    infect = infect - recovG - dead
    infect = infect[:299].reshape((299,1))
    recovGD = np.diff(recovG)
    recovGD - recovGD.reshape((299,1))

    gamma = np.linalg.lstsq(infect, recovGD)[0]
    return gamma

def approxRecovered(infect):
    recovG = np.zeros(len(infect))
    for i in range(len(infect) - 13):
        recovG[i + 13] = infect[i]
    return recovG


def getBeta(pop, q, infect, dead):
    
    sigma = regressionSlopeSeries(infect, 9 )
    recov = approxRecovered(infect)
    infectN = infect - recov - dead
    Z = infect
    Zt = q * Z
    It = q * infectN
    
    Beta = sigma * (q * pop + It[4:len(It)-4] - Zt[4:len(It)-4]) / (It[4:len(It)-4] * (q * pop - Zt[4:len(It)-4]))
    return Beta
    
    
def regressionSlopeSeries(points, window):
    """
    Given a series of points and a window size, estimates the slope at each point
    Outputs both the estimated slopes and regression errors for all applicable points
    Window should be odd, if it isn't it will automatically be interpreted as one larger than the argument passed
    """
    # cut the window in half so we know how long it extends on both sides
    half_window = window//2
    num_points = len(points)
    slopes = []
    for i in range(num_points):
        # if there are not enough previous values to satisfy window size, continue
        if i < half_window:
            continue
        # if there are not enough values after the current one to satisfy window size, break
        if num_points-i < half_window:
            break
        timeArray = np.ones((len(points), 2))
        timeArray[:,1]  = np.arange(len(points))
        pointArray = np.array(points)
        point_slope, res, _, _ = np.linalg.lstsq(timeArray, np.log(pointArray), rcond=None)
        slope = point_slope[1]
        slopes.append(slope)

    slopes = np.array(slopes)
    return slopes

In [12]:
pathc="../Data/Covid Tracking State Data/"
# modify the path syntax based on the current platform
if platform.system() == "Windows":
    pathc.replace("/", "\\")
    
filename = "IN.csv"

dates, infectRaw, recovRaw, deadRaw = SIRD_Model.loadData(pathc + filename)

pop = 3011000
numDays = 300
daysToPredict = 30
infect = infectRaw[0:numDays]
dead = deadRaw[0:numDays]

gamma = getGamma(infect,dead)
q = 0.0754
beta = getBeta(pop, q, infect, dead)
print(gamma, beta)

<ipython-input-11-19ec5a2f9d7e>:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  gamma = np.linalg.lstsq(infect, recovGD)[0]


ValueError: operands could not be broadcast together with shapes (293,) (292,) 